## Support vector machine usinf kernels

In [1]:
from zipfile import ZipFile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
%matplotlib inline

The inbuilt Support vector machine in sk-learn module provides both Linear SVC and SVC,i.e. for both linear classification and polynomial classification (using kernels). SVC provides many kernels linear(LinearSVC), rbf(radial basis function) and polynomial.<br>
<br>
'degree' refers to the degree of polynomial.<br>
'coef0' is the coeficient in the kernel trick, i.e. the dot product in lower dimension plus one raised to the power of the max polynomial term corresponds to the dot product in higher dimension. Here '1' is the coef0.<br>
'gamma' refers to the 1/2 square of s in the 'rbf' function.

In [2]:
def load(n):
    return np.genfromtxt('../Datasets ML/svm data/data{0}.txt'.format(str(n)))

def show_boundary(n,kernel='linear',c=1,s=1):
    plt.figure(figsize=(7,5))
    dataset=load(n)
    x=dataset[:,0:-1]
    y=dataset[:,-1]
    
    g=1/(2*(s*s))
    coef=1

    if kernel=='rbf':
        model=SVC(C=c,kernel='rbf',gamma=g)
        
    elif kernel=='poly':
        model=SVC(C=c,kernel='poly',degree=2,coef0=coef)
        
    else:
        model=SVC(C=c,kernel='linear')
    
    model.fit(x,y)
    sv=np.zeros(len(y),dtype=bool)
    sv[model.support_]=True
    nsv=np.logical_not(sv)
    
    x1_max=max(x[:,0])+1
    x1_min=min(x[:,0])-1
    x2_max=max(x[:,1])+1
    x2_min=min(x[:,1])-1
    
    x1=np.arange(x1_min,x1_max,0.05)
    x2=np.arange(x2_min,x2_max,0.05)
    xx1,xx2=np.meshgrid(x1,x2)
    
    plt.plot(x[(y==1)*sv,0],x[(y==1)*sv,1],'ro',markersize=10)
    plt.plot(x[(y==1)*nsv,0],x[(y==1)*nsv,1],'ro')
    plt.plot(x[(y==-1)*sv,0],x[(y==-1)*sv,1],'k^',markersize=10)
    plt.plot(x[(y==-1)*nsv,0],x[(y==-1)*nsv,1],'k^')
    
    val=np.c_[xx1.ravel(),xx2.ravel()]
    z=model.decision_function(val)
    
    for i in range(len(z)):
        
        z[i]=min(z[i],1)
        z[i]=max(z[i],-1)
        
        if (z[i]<1) & (z[i]>0):
            z[i]=0.5
            
        elif (z[i]<0) & (z[i]>-1):
            z[i]=-0.5
            
    plt.pcolormesh(xx1,xx2,z.reshape(xx1.shape),cmap=plt.cm.PRGn,vmin=-2,vmax=2)
    plt.show()
    

In [3]:
from ipywidgets import interact, interact_manual, IntSlider
interact_manual(show_boundary,c=[0.01,0.1,10,100,1000],s=IntSlider(2,2,40,3),kernel=['rbf','linear','poly'],
                n=IntSlider(1,1,5,1));

interactive(children=(IntSlider(value=1, description='n', max=5, min=1), Dropdown(description='kernel', index=…

## Kernel Perceptron for 'poly' and 'rbf'

In [4]:
def classifier(pt,x,y,alpha,b,kernel,s,deg):
    if kernel=='poly':
        k=(1+x.dot(pt.T))**deg
        val=y*alpha*k
        return np.sign(sum(val)+b)
    
    elif kernel=='rbf':
        gamma=1/2*((s*s))
        sim=(np.linalg.norm(x-pt,axis=1))**2
        k=np.exp(-(sim*gamma))
        val=y*alpha*k
        return np.sign(sum(val))
        
def kernel_perceptron(n,kernel='rbf',s=0.1,deg=2):
    data=load(n)
    x=data[:,0:-1]
    y=data[:,-1]
    
    iter=0
    n=x.shape[0]
    alpha=np.zeros(n)
    b=0
    converged=True
    done=False
    
    while not(done):
        done=True
        ind=np.random.permutation(n)
        
        for i in ind:
            label=classifier(x[i],x,y,alpha,b,kernel,s,deg)
            
            if y[i]!=label:
                done=False
                alpha[i]+=1
                b+=1
        iter+=1
            
        if iter>1000:
            done=True
            converged=False
            
    if converged:
        print('converged at {0}'.format(iter))
        
    else:
        print('Could\'nt converge at {0}'.format(iter))        
    
    x1_max=max(x[:,0])+1
    x1_min=min(x[:,0])-1
    x2_max=max(x[:,1])+1
    x2_min=min(x[:,1])-1
    
    x1=np.arange(x1_min,x1_max,0.05)
    x2=np.arange(x2_min,x2_max,0.05)
    xx1,xx2=np.meshgrid(x1,x2)
    
    plt.plot(x[(y==1),0],x[(y==1),1],'ro')
    plt.plot(x[(y==-1),0],x[(y==-1),1],'k^')
    
    val=np.c_[xx1.ravel(),xx2.ravel()]
    z=np.array([classifier(pt,x,y,alpha,b,kernel,s,deg) for pt in val])
            
    plt.pcolormesh(xx1,xx2,z.reshape(xx1.shape),cmap=plt.cm.PRGn,vmin=-2,vmax=2)
    plt.show()

In [11]:
from ipywidgets import interact, interact_manual, IntSlider
interact_manual(kernel_perceptron,s=[0.1,1,2,3,5],kernel=['rbf','poly'],n=IntSlider(1,1,5,1));

interactive(children=(IntSlider(value=1, description='n', max=5, min=1), Dropdown(description='kernel', option…

In [8]:
dat=np.genfromtxt('C:/Users/deshw/Downloads/Datasets ML/perceptron_at_work/data_1.txt')